# Extending RAG Engine Mini: A Practical Guide

This notebook demonstrates how to extend the RAG Engine Mini with a custom feature while maintaining educational quality and following architectural best practices.

## Learning Objectives

By the end of this notebook, you will:
1. Understand how to add a new feature following the existing architecture
2. Know how to implement a custom adapter following the ports and adapters pattern
3. Learn how to register your new component in the dependency injection container
4. Practice proper testing of your extension
5. Document your extension for educational purposes

In [ ]:
# Import necessary modules
import sys
import os
from pathlib import Path
from typing import List, Dict, Any
from dataclasses import dataclass
from abc import ABC, abstractmethod

# Add the project root to the path
project_root = Path("../../../")
sys.path.insert(0, str(project_root))

print(f"Project root: {project_root}")
print("Modules imported successfully")

## 1. Understanding the Architecture

Before extending the system, let's review the key architectural components we'll be working with:

In [ ]:
# Let's look at the source structure
src_path = project_root / "src"
print("Source structure:")
for item in sorted(src_path.iterdir()):
    if item.is_dir():
        print(f"  {item.name}/")
        sub_items = [sub_item.name for sub_item in item.iterdir() if sub_item.is_file()][:3]  # Show first 3 files
        if sub_items:
            print(f"    {', '.join(sub_items)}")
        print()

## 2. Defining Our Custom Feature

For this demonstration, we'll implement a **Query Intent Classifier** - a component that categorizes user queries to help route them to the most appropriate processing pipeline.

Our classifier will identify if a query is asking for:
- Factual information
- Comparative analysis
- Procedure/tutorials
- Unknown intent

This could help optimize the retrieval and generation process.

In [ ]:
# First, let's define our intent types
from enum import Enum

class QueryIntent(Enum):
    FACTUAL = "factual"
    COMPARATIVE = "comparative"
    PROCEDURAL = "procedural"
    UNKNOWN = "unknown"

# Define the port interface for our classifier
from typing import Protocol

class QueryIntentClassifierPort(Protocol):
    async def classify_intent(self, query: str) -> QueryIntent:
        """Classify the intent of a given query."""
        ...
        
# Define a data structure for our classification result
@dataclass
class IntentClassificationResult:
    query: str
    intent: QueryIntent
    confidence: float  # Confidence score between 0 and 1
    explanation: str  # Brief explanation of the classification

## 3. Implementing the Adapter

Now let's implement a simple classifier adapter that follows the ports and adapters pattern. For this example, we'll implement a rule-based classifier that can be easily understood and extended:

In [ ]:
# Implement a rule-based classifier
import re
import asyncio
from typing import Tuple

class RuleBasedQueryIntentClassifier:
    def __init__(self, default_confidence: float = 0.7):
        self.default_confidence = default_confidence
        
        # Define keywords for each intent type
        self.intent_keywords = {
            QueryIntent.FACTUAL: [
                r'what is', r'what was', r'who is', r'who was', r'when', r'define', 
                r'meaning of', r'explain', r'tell me about', r'how much', r'how many',
                r'give me information about', r'what does \w+ mean'
            ],
            QueryIntent.COMPARATIVE: [
                r'compare', r'vs', r'versus', r'difference between', r'similarities',
                r'better than', r'advantages of', r'disadvantages of', r'contrast',
                r'which is better', r'similar to', r'compared to'
            ],
            QueryIntent.PROCEDURAL: [
                r'how to', r'steps to', r'procedure for', r'process of',
                r'guide to', r'tutorial', r'instructions', r'make', r'build',
                r'implement', r'do', r'create', r'perform', r'execute'
            ]
        }
    
    async def classify_intent(self, query: str) -> IntentClassificationResult:
        """Classify the intent of a query using keyword matching and rules."""
        query_lower = query.lower().strip()
        
        # Check each intent type for keyword matches
        for intent, keywords in self.intent_keywords.items():
            for keyword_pattern in keywords:
                if re.search(keyword_pattern, query_lower):
                    explanation = f"Matched keyword pattern: '{keyword_pattern}'"
                    return IntentClassificationResult(
                        query=query,
                        intent=intent,
                        confidence=self.default_confidence,
                        explanation=exploration
                    )
        
        # If no specific intent matched, return unknown
        return IntentClassificationResult(
            query=query,
            intent=QueryIntent.UNKNOWN,
            confidence=0.1,  # Low confidence for unknown
            explanation="No specific intent patterns matched"
        )

# Verify our implementation satisfies the protocol
classifier = RuleBasedQueryIntentClassifier()
print("Classifier created successfully")
print(f"Type: {type(classifier)}")

## 4. Testing Our Implementation

Let's test our classifier with various types of queries:

In [ ]:
# Test the classifier with different query types
test_queries = [
    "What is machine learning?",
    "How to implement a neural network?",
    "Compare supervised and unsupervised learning",
    "Tell me about AI ethics",
    "Steps to deploy a model in production",
    "Which is better: SVM or Random Forest?",
    "Random question about weather"
]

# Run tests
print("Testing Query Intent Classifier:\n")
for i, query in enumerate(test_queries, 1):
    result = await classifier.classify_intent(query)
    print(f"{i}. Query: {result.query}")
    print(f"   Intent: {result.intent.value}")
    print(f"   Confidence: {result.confidence:.2f}")
    print(f"   Explanation: {result.explanation}")
    print()

## 5. Creating an Enhanced RAG Service

Now let's create an enhanced RAG service that uses our intent classifier to customize the retrieval process:

In [ ]:
# Create an enhanced RAG service that uses intent classification
from src.application.services.retrieval import RetrievalService
from src.adapters.vector.qdrant_adapter import QdrantAdapter
from src.adapters.llm.openai_adapter import OpenAIAdapter

class IntentAwareRAGService:
    def __init__(
        self,
        retrieval_service: RetrievalService,
        intent_classifier: QueryIntentClassifierPort,
        llm_adapter: Any  # Simplified for this example
    ):
        self._retrieval_service = retrieval_service
        self._intent_classifier = intent_classifier
        self._llm_adapter = llm_adapter
        
    async def ask_with_intent_awareness(self, query: str, k: int = 5):
        """Perform RAG with intent-aware optimizations."""
        # First, classify the intent
        intent_result = await self._intent_classifier.classify_intent(query)
        print(f"Query intent classified as: {intent_result.intent.value} (confidence: {intent_result.confidence:.2f})")
        
        # Adjust retrieval strategy based on intent
        adjusted_k = k
        if intent_result.intent == QueryIntent.FACTUAL:
            # For factual queries, we might want more focused results
            adjusted_k = max(3, k // 2)
            print(f"Adjusted k for factual query: {adjusted_k}")
        elif intent_result.intent == QueryIntent.COMPARATIVE:
            # For comparative queries, we might want more diverse results
            adjusted_k = min(10, k * 2)
            print(f"Adjusted k for comparative query: {adjusted_k}")
        elif intent_result.intent == QueryIntent.PROCEDURAL:
            # For procedural queries, we might want to prioritize instructional content
            print("Prioritizing instructional content for procedural query")
        
        # Perform the retrieval with the adjusted strategy
        chunks = await self._retrieval_service.retrieve(query, adjusted_k)
        
        # Generate the response
        # Note: This is simplified - in reality, you'd call the LLM with context
        response = f"Response generated with intent awareness. Intent: {intent_result.intent.value}, Chunks retrieved: {len(chunks)}"
        
        return {
            "response": response,
            "chunks": chunks,
            "intent_classification": intent_result,
            "adjusted_k": adjusted_k
        }

print("IntentAwareRAGService defined successfully")

## 6. Integration with Dependency Injection

Let's demonstrate how to register our new component in the dependency injection container:

In [ ]:
# Show how we would register our component in the container
# (This is illustrative - actual implementation would be in the bootstrap file)

from src.core.bootstrap import DIContainer

def demonstrate_container_registration():
    """Demonstrate how to register our new component in the DI container."""
    container = DIContainer()
    
    # Register our intent classifier
    container.register_singleton(QueryIntentClassifierPort, RuleBasedQueryIntentClassifier)
    
    # In a real implementation, we would also register our enhanced RAG service
    # container.register_transient(IntentAwareRAGService, IntentAwareRAGService)
    
    print("Components registered in container:")
    print("- QueryIntentClassifierPort -> RuleBasedQueryIntentClassifier")
    
    # Resolve the service
    resolved_classifier = container.resolve(QueryIntentClassifierPort)
    print(f"Resolved classifier type: {type(resolved_classifier).__name__}")
    
    return container

# Demonstrate the registration
container = demonstrate_container_registration()

## 7. Writing Tests for Our Extension

Let's create unit tests for our new component following the project's testing patterns:

In [ ]:
# Example test code (this would normally go in the tests/ directory)
import pytest
from unittest.mock import AsyncMock, MagicMock

async def test_intent_classifier_factual_query():
    """Test that factual queries are correctly identified."""
    classifier = RuleBasedQueryIntentClassifier()
    result = await classifier.classify_intent("What is machine learning?")
    
    assert result.intent == QueryIntent.FACTUAL
    assert result.confidence >= 0.5
    assert "what is" in result.explanation.lower()
    
    print("✓ Factual query test passed")

async def test_intent_classifier_procedural_query():
    """Test that procedural queries are correctly identified."""
    classifier = RuleBasedQueryIntentClassifier()
    result = await classifier.classify_intent("How to implement a neural network?")
    
    assert result.intent == QueryIntent.PROCEDURAL
    assert result.confidence >= 0.5
    assert "how to" in result.explanation.lower()
    
    print("✓ Procedural query test passed")

async def test_intent_classifier_unknown_query():
    """Test that unknown queries are handled properly."""
    classifier = RuleBasedQueryIntentClassifier()
    result = await classifier.classify_intent("Random text with no clear intent")
    
    assert result.intent == QueryIntent.UNKNOWN
    assert result.confidence <= 0.3  # Lower confidence for unknown
    
    print("✓ Unknown query test passed")

# Run our tests
await test_intent_classifier_factual_query()
await test_intent_classifier_procedural_query()
await test_intent_classifier_unknown_query()

## 8. Creating Educational Documentation

Now let's create documentation for our extension to maintain the educational quality of the project:

In [ ]:
# Generate documentation for our extension
documentation = f"""
# Query Intent Classification Extension

## Overview

The Query Intent Classification extension adds the ability to categorize user queries by their intent before processing them. This allows for intent-aware optimizations in the RAG pipeline.

## Architecture

The extension follows the ports and adapters pattern:

- **Port**: `QueryIntentClassifierPort` defines the interface
- **Adapter**: `RuleBasedQueryIntentClassifier` provides the implementation
- **Service**: `IntentAwareRAGService` uses the classification to optimize processing

## Intent Categories

The classifier recognizes these intent types:

- `{QueryIntent.FACTUAL.value}`: Queries seeking factual information
- `{QueryIntent.COMPARATIVE.value}`: Queries comparing different concepts
- `{QueryIntent.PROCEDURAL.value}`: Queries seeking instructions/processes
- `{QueryIntent.UNKNOWN.value}`: Queries that don't match other categories

## Usage

```python
# Example usage
classifier = RuleBasedQueryIntentClassifier()
result = await classifier.classify_intent("What is machine learning?")
print(f"Intent: {{result.intent.value}}")
print(f"Confidence: {{result.confidence}}")
```

## Benefits

1. **Optimized Retrieval**: Different query intents may benefit from different retrieval strategies
2. **Enhanced Response**: Knowing the intent allows for tailored response generation
3. **Analytics**: Track which types of queries are most common

## Possible Improvements

1. Replace the rule-based approach with a machine learning model
2. Add more sophisticated NLP techniques
3. Create intent-specific prompt templates
"""

# Save documentation to a file
docs_path = project_root / "docs" / "educational" / "query_intent_classification_extension.md"
with open(docs_path, 'w', encoding='utf-8') as f:
    f.write(documentation.strip())

print(f"Documentation saved to: {docs_path}")
print("Documentation preview:")
print(documentation.strip()[:500] + "...")  # Show first 500 chars

## 9. Creating an ADR (Architecture Decision Record)

Let's document our architectural decision with an ADR:

In [ ]:
# Create an ADR for our extension
adr_content = f"""
# ADR 006: Query Intent Classification for RAG Optimization

## Context

Different types of queries may benefit from different retrieval and generation strategies. 
A "one-size-fits-all" approach to RAG processing may not be optimal for all query types.

For example:
- Factual queries might benefit from focused retrieval
- Comparative queries might benefit from diverse retrieval
- Procedural queries might benefit from instructional content prioritization

## Decision

We will implement a Query Intent Classification system that:
1. Categorizes incoming queries into intent types (factual, comparative, procedural, unknown)
2. Adjusts the RAG pipeline based on the detected intent
3. Follows the ports and adapters pattern for extensibility
4. Provides confidence scores for transparency

## Approach

1. Create a `QueryIntentClassifierPort` interface
2. Implement a rule-based classifier as the initial implementation
3. Create an enhanced RAG service that uses intent classification
4. Register the component in the dependency injection container
5. Provide comprehensive tests and documentation

## Consequences

### Positive
- More tailored responses for different query types
- Potential performance improvements through optimized retrieval
- Extensible design allows for more sophisticated classification later
- Educational example of adding features to the RAG engine

### Negative
- Additional computational overhead for intent classification
- Complexity increase in the RAG pipeline
- Risk of misclassification leading to suboptimal processing

## Alternatives Considered

1. **No Intent Classification**: Keep the current generic approach - simpler but less optimized
2. **ML-Based Classification**: Use a trained model for classification - more accurate but more complex
3. **Hybrid Approach**: Combine rule-based and ML approaches - most flexible but most complex

## Status

Proposed → Accepted

## Implementation Notes

The initial implementation uses a rule-based approach for educational purposes and simplicity. 
Future iterations could replace this with a machine learning model while maintaining the same interface.
"""

# Save ADR to a file
adr_path = project_root / "docs" / "adr" / "006-query-intent-classification.md"
with open(adr_path, 'w', encoding='utf-8') as f:
    f.write(adr_content.strip())

print(f"ADR saved to: {adr_path}")
print("ADR preview:")
print(adr_content.strip()[:500] + "...")  # Show first 500 chars

## 10. Summary and Next Steps

In this notebook, we demonstrated how to extend the RAG Engine Mini with a custom feature while following best practices:

In [ ]:
summary_points = [
    "Defined a clear interface (port) for our new feature",
    "Implemented the feature following the existing architectural patterns",
    "Created comprehensive tests for our implementation",
    "Integrated our component with the dependency injection system",
    "Documented our extension with educational materials",
    "Created an ADR to record our architectural decisions",
    "Ensured our extension maintains the educational quality of the project"
]

print("Summary: Key steps for extending the RAG Engine Mini:")
for i, point in enumerate(summary_points, 1):
    print(f"{i}. {point}")

print("\nThe extension is now ready to be integrated into the full codebase!")

next_steps = [
    "Move the implementation code to the appropriate files in src/",
    "Add the component to the main dependency injection container",
    "Write additional tests covering edge cases",
    "Implement a more sophisticated ML-based classifier as an alternative",
    "Add the feature to the API layer with appropriate endpoints",
    "Create a notebook demonstrating the feature in action"
]

print("\nNext steps to fully integrate this extension:")
for i, step in enumerate(next_steps, 1):
    print(f"{i}. {step}")